## Problem Definition
We build a program that can play video games:
```
Input: Game-Environment (env)
Output: Game-Policy (policy)
```

__env__:
```
Input: action (0~k, say, 2)
Output: screen-image, reward, game-is-over
```

__policy__:
```
Input: screen-image
Output: action
```

It is not difficult to imagine how a policy “plays” an env. The goal is to design the policy self-inspection and adjustment scheme (call this meta-policy if you like), so the total reward it receives in a game maximises. Note this setting can be more generic than you might think of:
- the game can give a reward of any constant positive value at each step to simply encourage the player to stay playing as long as possible, which makes sense in some balancing or jumpping games. In some games that can even mean disencourage winning, such as in ball games!
- the game can give a reward of any constant negative value at each step to encourage quick playing, such as in a maze game without suicidal option, this is equal to saying "hurry up!"

## Building Blocks

1. Neural Networks
    1. Network building using torch
    2. Network training using gradient descent
        1. Compute gradients with back propagation
        2. Commit parameter update along gradient direction
        
2. Reinforcement Learning Algorithms
    1. Long-term evaluation of actions -- building a slot-machine playing agent
    2. Handling machines with internal states
    3. Handling machines with MANY internal states
        1. Using neural networks to estimate action values
        2. Adjust action probability by reviewing consequences

## Technical Terms

A small number of technical terms are used in our discussion. One way to treat a strange jargon is just ignore it when encountering, and let its meaning emerge by itself during your study. If you find your short-term memory is going to explode because the need to keep track many strange notions -- it might be helpful to look up in a glossary such as [here](https://www.analyticsvidhya.com/glossary-of-common-statistics-and-machine-learning-terms/). Or simply Google the new concept. However, I am afraid google/wiki-def of the notion can only partially help your study -- the key is the fact that many new concepts need to be learned, so only careful review can help achieve a deep understanding.




## Code

You should be able to handle the homeworks left in the prior studio. E.g. after we have studied the "exploitation and exploration" in slot-machine playing. You should be able to construct a slot-machine player, from the template we discussed on class.

Most importantly, you need to be able to modify and test code template provided to you.

## Math Glossary

You need to understand the relationship between matrix-vector production and its correspondence to linear equation systems; derivation of simple computations and common functions, $d e^x/dx = e^x$ for example.

Some math representation of useful concepts are:

- $s_t$: the state observed at time $t$. This is usually but NOT always the stuff returned to the agent at the time taking actions. The most prominent exception is the screen-image-based states in our video-game playing examples. One applies some simple preprocessing to the states.

- $a_t$: the action taken at time $t$. Generally, it is an integer $\{0, 1, ..., K-1\}$ if there are $K$ different actions. Keep in mind that the actual action could be represented differently, such as "press A-button". For a decision making agent, given all possible action choices, choosing actions is eqivalent to choosing the indexes.

- $r_t$: the immediate reward received at time $t$. Note some authors used to let $r_t$ refer to the reward received __after__ taking action $a_t$ in state $s_t$, while others take $r_t$ as the reward received __at the beginning__ at time $t$, after taking action $a_{t-1}$ in state $s_{t-1}$. In whatever way, the procedure: in $s_t$ taking action $a_t$ according to some policy $\pi$ arriving the next state $s_{t+1}$ and receiving a reward $r_{t}$ (or $r_{t+1}$ subject to your choice of denotation) is called a __transition step__.

- $\pi(\cdot|s)$, given the state $s$, a policy $\pi(a|s)$ (not to be confused with the $\pi\approx3.1416$) assigns a non-negative real number to each action -- it dictates the possibility of choosing the action given $s$. If a policy is deterministic, rather than stochastic, it can attribute all probabilities to one particular action, so that the corresponding $\pi(a|s)=1$ and $\pi(a'|s)=0$ for all other actions $a'$.

- $Q^\pi(s, a)$, evaluation of __long term__ return for taking action $a$ in state $s$. Since it considers future effects, it relies on the on-going policy, $\pi$. Note taking $a$ at the current state $s$, the very first step of this evaluation is not necessarily with respect to $\pi$. Consider this $Q$-evaluation as answering a hypothetical question: what the long term reward would have been if she took action $a$ at $s$ and followed $\pi$ henceafter. Of course if this evaluation is known, it is wise to take the action maximising this $Q$ at each $s$.

- Note in neural network implementation, $Q$- and $\pi$-nets share the same structure: map states to $K$ numbers, where $K$ is the number of actions.




## Code Style

We follow python code style specified by [PEP8](https://www.python.org/dev/peps/pep-0008/). You can install PyCharm (free for students) to help you ensure your code complies.

Good code should be self-explaining for technical operations, e.g.

```
action_probabilities = policy(state)
rand_hit = rand()
accumulated_probability = 0
for a_, p_ in zip(actions, action_probabilities):
    accumulated_probability += p_
    if rand_hit <= accumulated_probability:
        chosen_action = a_
        break
```

The individual steps should be clear enough. Comments explaining the design logic may be added if that code is an essential part
```
# Randomly choose an action according to the policy.
# rand_hit is in [0, 1], we start from 0, accumulating the probability of 
# one action after another, at the moment rand_hit is exceeded, that action
# is our choice.
```


## Useful libraries

Mainly we will use ```numpy, pytorch, gym``` for the core functionalities. Several often used utility libraries includes ```PIL, os, collections, itertools, matplotlib, time```. Note some libraries come with sub-modules that need independent import, such as ```torch.nn```. 

Some libraries are imported for some particular purpose, e.g. you can save a collection of parameters in a dictionary as a file using ```json```. However, the functionality can be easily replaced using ```torch.save```. Those are non-essential dependencies, you can choose not to use those libraries if they cause trouble on a particular computer environment.

## Net-building in torch

Pytorch provides convenient building blocks for deep neural networks. All processing stages that accept information in some predefined form and output in another predefined form are of the same kind: `module`. Note the notion definition is recursive. If we combine two modules, A and B, so that the information is processed by A, while the middle product is fed to B producing the final product, the combined operation "Process by A then B" is itself a processing step albeit a more complex one, and therefore belongs to the `module` class as well. In this way, one can make complex, reusable modules.

Pytorch contains many commonly used modules:
- [linear modules](http://pytorch.org/docs/master/nn.html#linear-layers) for generic mapping, it is the rule-of-thumb choice when you know the input data has $m$ attributes and the output has $n$.
- [convolutional modules](http://pytorch.org/docs/master/nn.html#convolution-layers) for dealing image data

# Policy Gradient Example

In [1]:
%run a3c.py

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
=> no checkpoint found at 'True'
Starting 0
Starting 1
Starting 2


KeyboardInterrupt: 

```
# this is what happened at time t
v = net(im)
v[0], v[1], v[2]
0.9   0.08  0.02
a is 0

at T: we have reviewed the episode, and found: 
at t, we were doing better than average

what is the value you hope v to have next time if we observed im:
v[0] v[1] v[2]
0.92 0.05, 0.03

```
so we want to change net.parameters, so to get action 0 (the action I took at time $t$, that means I need to keep record of actions)

$\nabla_\theta \log p(s, a)$ -- the direction along which I adjust net.parameters `log net(im)[0]` will increase.

$\nabla_\theta \log p(s, a) A(s, a) $